## Text Summarization using fine tuned facebook/bart-base model
---



***Install necessary Libraries***

In [2]:
!pip install datasets evaluate transformers rouge-score nltk

In [3]:
!pip install transformers[torch]

In [90]:
!pip install nltk

***Login to Huggingfacehub***

In [4]:
from huggingface_hub import notebook_login

notebook_login()

***Installing Git-lfs***

In [5]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


***Checking Transformer Version***

In [6]:
import transformers

print(transformers.__version__)

4.38.2


In [7]:
import pandas as pd
import numpy as np

In [91]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

### Read and PreProcess the dataset for Finetuning the base model

In [8]:
input_df = pd.read_csv('curated_data_subset.csv')
input_df.head()

,Unnamed: 0,title,summary,url,date,article_content
0,0,Tencent gains approval to sell mutual funds to...,Tencent has been granted a licence from the Ch...,http://www.scmp.com/business/companies/article...,49:58.9,Traditional finance houses now being seriously...
1,1,"India testing blockchains in education, health...",India is testing blockchain applications in ed...,https://www.vccircle.com/niti-aayog-explores-b...,33:08.3,"The Indian government's policy think tank, Nit..."
2,2,Higher living wage risks robot takeover of low...,The UK Institute for Fiscal Studies has warned...,https://news.sky.com/story/ifs-living-wage-inc...,32:10.2,Increases in minimum wage levels risk raising ...
3,3,Regus WeWork may seek stock-market flotation t...,Co-working start-up WeWork may go public this ...,https://www.fool.com/investing/2018/01/03/will...,31:59.6,"Depending on whom you ask, WeWork is either a ..."
4,4,AMD poised to gain market share as Intel pound...,Intel has seen more than $11bn wiped off its m...,https://www.cnbc.com/2018/01/05/amd-is-big-win...,17:13.3,AMD is big winner from chip flaw fiasco as mor...


In [9]:
input_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       100 non-null    int64 
 1   title            100 non-null    object
 2   summary          100 non-null    object
 3   url              100 non-null    object
 4   date             100 non-null    object
 5   article_content  99 non-null     object
dtypes: int64(1), object(5)
memory usage: 4.8+ KB


In [10]:
input_df.isnull().sum()

Unnamed: 0         0
title              0
summary            0
url                0
date               0
article_content    1
dtype: int64

In [11]:
input_df.dropna(inplace=True, ignore_index= True)

In [12]:
input_df.isnull().sum()

Unnamed: 0         0
title              0
summary            0
url                0
date               0
article_content    0
dtype: int64

In [13]:
input_df.loc[29, :]

Unnamed: 0                                                        30
title              Fluenta's new gauges measure gas at extreme te...
summary            Fluenta has launched two ultrasonic flow trans...
url                https://www.oilfieldtechnology.com/product-new...
date                                                         20:05.4
article_content    Fluenta has announced the launch of its new ra...
Name: 29, dtype: object

In [14]:
input_df.rename(columns = {'article_content':'Text', 'summary':'Summary'}, inplace=True)

In [15]:
input_df = input_df[['Text', 'Summary']]

In [16]:
input_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Text     99 non-null     object
 1   Summary  99 non-null     object
dtypes: object(2)
memory usage: 1.7+ KB


In [17]:
input_df.head(10)

,Text,Summary
0,Traditional finance houses now being seriously...,Tencent has been granted a licence from the Ch...
1,"The Indian government's policy think tank, Nit...",India is testing blockchain applications in ed...
2,Increases in minimum wage levels risk raising ...,The UK Institute for Fiscal Studies has warned...
3,"Depending on whom you ask, WeWork is either a ...",Co-working start-up WeWork may go public this ...
4,AMD is big winner from chip flaw fiasco as mor...,Intel has seen more than $11bn wiped off its m...
5,"MUMBAI: About 250 companies, including some of...",Demand for cyber insurance among Indian compan...
6,Public sector organisations have quadrupled th...,The UK's public sector has increased its spend...
7,A worker pulls carts full of customer orders a...,The average price for large industrial plots o...
8,"Office take up in quarter four, 2017 in the ce...",Office take-up in Birmingham during 2017 broke...
9,Elevate your enterprise data technology and st...,"Machine learning chatbots – such as Ada, which..."


### Split the data into Train, Validation and Test dataset

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_val = train_test_split(input_df, test_size = 0.2)

X_train, X_test = train_test_split(X_train, test_size = 0.1)

In [19]:
print(f'Shape of X_train is: {X_train.shape}')
#print(f'Shape of y_train is: {y_train.shape}')

print(f'Shape of X_val is: {X_val.shape}')
#print(f'Shape of y_val is: {y_val.shape}')

print(f'Shape of X_test is: {X_test.shape}')
#print(f'Shape of y_test is: {y_test.shape}')

Shape of X_train is: (71, 2)
Shape of X_val is: (20, 2)
Shape of X_test is: (8, 2)


***Saving the dataset for training, validation and test***

In [20]:
input_df.to_csv('curated_dataset_for_summarization.csv', index=False)
X_train.to_csv('training_data.csv', index=False)
X_val.to_csv('validation_data.csv', index=False)
X_test.to_csv('test_data.csv', index=False)

### Loading the dataset from Huggingface Dataset

In [21]:
from datasets import load_dataset

data_files = {"train": "training_data.csv", "val":"validation_data.csv", "test": "test_data.csv"}
dataset = load_dataset("sudhanshusinghaiml/curated-dataset-for-summarization", data_files= data_files)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [22]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Text', 'Summary'],
        num_rows: 71
    })
    val: Dataset({
        features: ['Text', 'Summary'],
        num_rows: 20
    })
    test: Dataset({
        features: ['Text', 'Summary'],
        num_rows: 8
    })
})

***Displaying data from loaded dataset***

In [23]:
dataset['train'][29]

{'Text': '"But, on the other hand, we would be foolish to rule anything out. We know that Asia-Pacific will be a very important market and we know a lot of the global growth in the future will come from there."',
 'Summary': 'According to the UK\'s International Trade Secretary, Liam Fox, the UK could feasibly join the Trans-Pacific Partnership (TPP), saying "it would be foolish to rule anything out". The organisation is made up of Australia, Mexico, New Zealand, Canada, Chile, Japan, Singapore, Brunei, Peru, Vietnam and Malaysia - with Donald Trump pulling the US out last year - and\xa0is currently in renegotiation under the new name of the Comprehensive and Progressive Agreement for Trans-Pacific Partnership. Its aims are to\xa0lower both non-tariff and tariff barriers to trade and to provide a forum to settle international disputes.'}

In [24]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [25]:
show_random_elements(dataset["train"])

In [26]:
from evaluate import load

metric = load("rouge")

In [27]:
metric

EvaluationModule(name: "rouge", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id=None)}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLsum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/

### Tokenize the training, Validation and test dataset from AutoTokenizer

***Creating Model Checkpoint for better readability***

In [28]:
model_checkpoint = 'facebook/bart-base'
max_input_length = 1024
max_target_length = 128
prefix = "Summarize: "

In [29]:
from transformers import BartTokenizer

tokenizer = BartTokenizer.from_pretrained(model_checkpoint)

In [81]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["Text"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["Summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [82]:
preprocess_function(dataset['train'][:2])

{'input_ids': [[0, 38182, 3916, 2072, 35, 29352, 1787, 138, 22226, 7219, 36, 347, 42456, 43, 892, 7441, 14, 5, 194, 9, 5993, 16, 5, 21084, 194, 11, 5, 382, 7, 4318, 8518, 4, 50118, 50118, 30219, 2593, 4481, 3441, 10, 319, 9, 3459, 476, 8, 5, 476, 1162, 10356, 11, 358, 194, 4, 50118, 50118, 20930, 15, 18694, 6335, 17, 27, 29, 665, 892, 9, 5, 701, 9, 11196, 4481, 420, 5, 382, 6, 24, 16, 855, 21084, 7, 4318, 8518, 11, 5993, 480, 4382, 1042, 23, 361, 4, 5677, 3205, 228, 42738, 4650, 5, 674, 701, 9, 4481, 65, 8518, 23, 68, 246, 6, 28835, 4, 50118, 50118, 713, 16, 3625, 7246, 87, 5, 595, 425, 9, 8518, 6, 61, 16, 855, 1446, 23, 198, 68, 1092, 6, 151, 228, 12911, 6, 25, 9, 1228, 86, 4, 50118, 50118, 13841, 1493, 11, 5, 382, 16, 24, 6162, 7, 4318, 116, 50118, 50118, 1121, 49, 892, 6, 18694, 6335, 67, 2319, 5, 701, 9, 8518, 4481, 716, 15, 5, 42738, 1580, 4850, 9, 5, 130, 144, 1406, 4481, 19880, 6, 13953, 6, 5, 3702, 448, 5101, 208, 466, 6, 5, 3702, 448, 5101, 208, 406, 6, 8, 5, 35383, 231, 6, 25

In [83]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/71 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

### Train or Fine Tune with Pytorch Trainer on new dataset

We can download the pretrained model and fine-tune it. Since our task is of the sequence-to-sequence kind, we use the AutoModelForSeq2SeqLM class. Like with the tokenizer, the from_pretrained method will download and cache the model for us

In [84]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [85]:
from accelerate import Accelerator

accelerator = Accelerator()

batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    "sudhanshusinghaiml/facebook-bart-base-finetuned",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True
    # push_to_hub=True
)

We need a special kind of data collator, which will pad the inputs to the maximum length in the batch for all datasets and texts as well as summary

In [86]:
dataCollator = DataCollatorForSeq2Seq(tokenizer, model=model)

The last thing to define for our Seq2SeqTrainer is how to compute the metrics from the predictions. We need to define a function for this, which will just use the metric we loaded earlier, and we have to do a bit of pre-processing to decode the predictions into texts:

In [87]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

Then we just need to pass all of this along with our datasets to the Seq2SeqTrainer:

In [88]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["val"],
    data_collator=dataCollator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [92]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,2.758328,15.655800,4.413800,11.243500,13.705000,20.000000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch,Training Loss,Validation Loss


TrainOutput(global_step=5, training_loss=3.1713235855102537, metrics={'train_runtime': 7.8909, 'train_samples_per_second': 8.998, 'train_steps_per_second': 0.634, 'total_flos': 84023351193600.0, 'train_loss': 3.1713235855102537, 'epoch': 1.0})

In [94]:
# Save the model
trainer.save_model('facebook-bart-base-finetuned')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


Test the Preiction of the model

In [98]:
prediction_results = trainer.predict(tokenized_datasets['test'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [109]:
# Let us pull the model from huggingface


config.json:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/262 [00:00<?, ?B/s]

In [110]:
def summarize_from_finetuned_model(article):
    # Load BART model and tokenizer
    model_name = 'sudhanshusinghaiml/facebook-bart-base-fintuned'
    tokenizer = BartTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

    # Tokenize and encode the article
    inputs = tokenizer(article, return_tensors='pt', max_length=1024, truncation=True)

    # Generate summary
    summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=128, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return summary

# Example usage
article = """
Former U.S. President Donald Trump charged in classified documents probe. There was no immediate confirmation from the Justice Department regarding Mr. Trump’s assertion, although some U.S. media outlet cited sources saying that the former U.S. President has bee indicted.
"""

summary = summarize_from_finetuned_model(article)
print("Summary:")
print(summary)

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/999k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

Summary:
Former U.S. President Donald Trump charged in classified documents probe. There was no immediate confirmation from the Justice Department regarding Mr. Trump


In [112]:
article = """Packing all the contextual information from a document into a short summary is much harder with long text. If our summary has to be say 5 sentences max it is much harder to
decide what information is valuable enough to be added with 500 words vs 50,000 words. Chunking algorithms are often required, but they do grow the data variance coverage a model must have
to be accurate. Chunking algorithms control how much of the larger document we pass into a summarizer based on the max tokens the model allows and parameters we’ve set. The new dynamic
nature of the input data means our data variance is much larger than what is seen with smaller text. Longer documents often have much more internal data variance and swings in the information.
Use causes such as blog posts, interviews, transcripts and more have multiple swings in the dialog that make it harder to understand what contextual information is valuable for the summary.
Models have to learn a much deeper relationship between specific keywords, topics, and phrases as the text grows. There are two main types of summarization that are used as the baseline for
any enhanced versions of summarization - Extractive and abstractive. They focus on how the key information found in the input text is reconstructed in the generated summary in their own ways.
Both of these methods have their own unique challenges that pop up when looking at using longer text."""

summary = summarize_from_finetuned_model(article)
print("Summary:")
print(summary)

Summary:
Packing all the contextual information from a document into a short summary is much harder with long text. If our summary has to be say
